# Hadoop and MapReduce

In [62]:
! pwd

/home/azureuser


## Основы MapReduce

В своей сути MapRedcue это очень простая парадигма. Допустим у нас есть датасет

In [63]:
! curl https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv > tweets_1.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.9M  100 89.9M    0     0  20.1M      0  0:00:04  0:00:04 --:--:-- 27.7M


In [64]:
! head -n 2 tweets_1.csv

Мы хотим в этом датасете что-нибудь найти. Например (сейчас будет баян), посчитать количество уникальных слов. Мы могли бы сделать что-то такое:

#### Вариант 1 
Используем исключительно питон

In [65]:
! pip install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [66]:
%%time
%%memit

from collections import Counter
import csv
import re
import sys

counter = Counter()
pattern = re.compile(r"[a-z]+")

with open('tweets_1.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            counter[word] += 1

for word, count in counter.most_common(10):
    print(f"{word}\t{count}")

t	268703
co	250375
https	221366
the	69350
to	55972
a	43420
in	37099
s	36085
of	33579
http	28661
Memory 19470488
peak memory: 90.38 MiB, increment: 9.68 MiB
CPU times: user 10.6 s, sys: 1.27 s, total: 11.9 s
Wall time: 14.2 s


Такое сработает только если у нас не очень много данных и они все вмещаются в оперативную память

In [68]:
! du -h tweets_1.csv

90M	tweets_1.csv


#### Вариант 2
Используем парадигму Map Reduce

В этом примере у нас всего 90 мегабайт данных, и на моем компьютере они обрабатываются за примерно 30 секунд с помощью питона. Теперь представим (это достаточно несложно), что у нас приходит новых данных приходит _десятки терабайт_ в сутки. Такое уже не поместится ни в один сервер, поэтому нам нужно придумать что-нибудь похитрее.

MapReduce как раз является парадигмой, помогающей обрабатывать большие объемы данных, за счет простоты своего устройства.

Приятная новость - для того, чтобы понять и научиться программировать программы в парадигме MapReduce вам потребуется... **5 секунд!**

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/you-know-mapreduce.png" width="400">

Все потому что вы уже прошли семинар по Bash и научились составлять большие программы в виде компоновки небольших  программ, соединенных пайпами. По своей сути программа на MapReduce - это хорошо отмасштабированная программа вида.

```bash
cat data.txt | map | sort | reduce
```

Сортировку за вас выполняет сам фреймворк (и ее вы можете дополнительно настроить точно такое как и команду sort). А также он самостоятельно разобьем данные на части и параллельно запустит операции map и reduce. 

Таким образом на самом деле Hadoop - это всего лишь гигантская машина сортировки, которая дополнительно дает вам некоторые гарантии:

* Для всех данных параллельно будет применена операция map
* Данные будут отсортированы по указанному вами ключу
* Каждый ключ будет целиком передан на один и только один reduce

Программисту остается реализовать программу, которая состоит из двух компонент: `map` и `reduce`. 

Операция `map` -- это просто функция из одного элемента в другой элемент, у которого есть первичный ключ. 

Операция `reduce` -- это коммутативная и ассоциативная агрегация всех элементов по ключу. Чтобы эти операции совершить, надо разбить весь вход на куски данных и отправить их на машины, чтобы они выполнялись в параллель, а весь выход операции map идёт в операцию shuffle, которая по одним и тем же ключам определяет записи на одинаковые хосты. 

В итоге получается, что мы можем спокойно увеличивать количество worker'ов для map операций и с увеличением количества данных мы лишь будем линейно утилизировать количество машин, то же самое с операцией reduce -- мы можем добавлять машины с ростом увеличения количества ключей линейно, не боясь того, что мы не можем позволить на одной какой-то машине больше памяти или диска.

Давайте напишем маппер и редьюсер на питоне для этой задачи:

In [69]:
%%writefile wordcount.py
import sys
import csv
import re


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv.reader(iter(sys.stdin.readline, '')):
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    word, number = next(sys.stdin).split('\t')
    number = int(number)
    for line in sys.stdin:
        current_word, current_number = line.split('\t')
        current_number = int(current_number)
        if current_word != word:
            print("{}\t{}".format(word, number))
            word = current_word
            number = current_number
        else:
            number += current_number
    print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing wordcount.py


Важно еще удалить голову у таблицы, иначе подсчеты могут быть некоректными

In [70]:
! sed -i -e '1'd tweets_1.csv

In [72]:
! cat tweets_1.csv | python wordcount.py map | sort -k1,1 | head

a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
sort: write failed: 'standard output': Broken pipe
sort: write error


In [73]:
%%time

! cat tweets_1.csv | \
    tqdm --total $(cat tweets_1.csv | wc -l)| \
    python wordcount.py map | \
    sort -k1,1 | \
    python wordcount.py reduce > result.txt

100%|█████████████████████████████████| 243891/243891 [00:26<00:00, 9377.95it/s]
CPU times: user 565 ms, sys: 67.9 ms, total: 633 ms
Wall time: 32.5 s


In [74]:
! head result.txt

a	43420
aa	151
aaa	13
aaaaaa	1
aaaaaaaaaaaaand	1
aaaaaaaaaall	1
aaaaaaaamen	1
aaaaaaaand	2
aaaaaaargh	1
aaaaaand	2


Отлично! Слова есть, осталось только найти top-10.

In [75]:
%%writefile top10.py
import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(10):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing top10.py


In [76]:
! cat result.txt | \
    tqdm --total $(cat result.txt | wc -l) | \
    python top10.py map | \
    sort -t'+' -k2,2nr -k1,1 | \
    python top10.py reduce > top-10.txt

100%|███████████████████████████████| 346613/346613 [00:01<00:00, 275994.94it/s]


In [77]:
! cat top-10.txt

t	268703
co	250375
https	221366
the	69350
to	55972
a	43420
in	37099
s	36085
of	33579
http	28661


На MapReduce мы задачу переписали, однако быстрее работать она пока не стала. Все дело в том, что мы это еще не на кластере запускали! Время запускать все на настоящем кластере!

## Azure

Сначала давайте сохраним где-нибудь наш пароль от ажура

In [6]:
import pathlib
import getpass
password = getpass.getpass()

with pathlib.Path('~/.azure_password').expanduser().open('w') as file:
    file.write(password + '\n')

%cat ~/.azure_password | head -c 1

········
9

In [7]:
%cd cloud-configuration

/Users/naorlov/Dropbox/hse/teaching/2020-lsml/lsml-2021-internal/cloud-configuration


In [ ]:
%%writefile hadoop.tf

resource "azurerm_storage_container" "lsml_hadoop_sc" {
  name                  = "lsmlhdinsight"
  storage_account_name  = azurerm_storage_account.lsml_sa.name
  container_access_type = "private"
}

resource "azurerm_hdinsight_hadoop_cluster" "lsml_hc" {
  name                = "lsml-hdicluster"
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location            = azurerm_resource_group.lsml_rg.location
  cluster_version     = "4.0"
  tier                = "Standard"

  component_version {
    hadoop = "3.1"
  }

  gateway {
    enabled  = true
    username = "azureuser"
    password = "Password123!"
  }

  storage_account {
    storage_container_id = azurerm_storage_container.lsml_hadoop_sc.id
    storage_account_key  = azurerm_storage_account.lsml_sa.primary_access_key
    is_default           = true
  }

  roles {
    head_node {
      vm_size  = "A5"  # 2 cpu 4 ram
      username = "azureuser"
      password = "Password123!"
    }

    worker_node {
      vm_size               = "Standard_D12_V2" # 4 cpu 28 ram
      username              = "azureuser"
      password              = "Password123!"
      target_instance_count = 2
    }

    zookeeper_node {
      vm_size  = "Standard_A2_V2"  # 2 cpu 4 ram
      username = "azureuser"
      password = "Password123!s"
    }
  }
}

output "ssh_endpoint" {
    value = azurerm_hdinsight_hadoop_cluster.lsml_hc.ssh_endpoint
}

output "https_endpoint" {
    value = azurerm_hdinsight_hadoop_cluster.lsml_hc.https_endpoint
}

In [23]:
%%writefile common.tf

provider "azurerm" {
  version = "=2.40.0"
  features {}
}

resource "azurerm_resource_group" "lsml_rg" {
  name = "lsml-resource-group"
  location = "westus"
}

resource "azurerm_virtual_network" "lsml_vn" {
  name = "lsml-vitrual-network"
  resource_group_name = azurerm_resource_group.lsml_rg.name
  location = azurerm_resource_group.lsml_rg.location
  address_space = ["10.0.0.0/16"]   # Пул адресов внутри сети
}

resource "azurerm_storage_account" "lsml_sa" {
  name                     = "lsmlhdinsightstore"
  resource_group_name      = azurerm_resource_group.lsml_rg.name
  location                 = azurerm_resource_group.lsml_rg.location
  account_tier             = "Standard"
  account_replication_type = "LRS"
}

Overwriting common.tf


In [11]:
! ls

common.tf  hadoop.tf


In [8]:
! pwd

/Users/naorlov/Dropbox/hse/teaching/2020-lsml/lsml-2021-internal/cloud-configuration


In [9]:
! terraform init

zsh:1: command not found: terraform


In [19]:
! wget https://aka.ms/InstallAzureCLIDeb -O - | sudo bash

--2021-01-06 15:04:36--  https://aka.ms/InstallAzureCLIDeb
Resolving aka.ms (aka.ms)... 23.53.53.76
Connecting to aka.ms (aka.ms)|23.53.53.76|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://azurecliprod.blob.core.windows.net/$root/deb_install.sh [following]
--2021-01-06 15:04:37--  https://azurecliprod.blob.core.windows.net/$root/deb_install.sh
Resolving azurecliprod.blob.core.windows.net (azurecliprod.blob.core.windows.net)... 13.88.145.64
Connecting to azurecliprod.blob.core.windows.net (azurecliprod.blob.core.windows.net)|13.88.145.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4309 (4.2K) [text/x-sh]
Saving to: ‘STDOUT’

-                   100%[===================>]   4.21K  --.-KB/s    in 0s      

2021-01-06 15:04:38 (309 MB/s) - written to stdout [4309/4309]

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [109 kB]    

In [20]:
! az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DF3WP52LX to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "21f26c24-0793-4b07-a73d-563cd2ec235f",
    "id": "7d1225ca-27cc-40b7-8036-c62a48072ba8",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Спонсорское предложение Microsoft Azure 2",
    "state": "Enabled",
    "tenantId": "21f26c24-0793-4b07-a73d-563cd2ec235f",
    "user": {
      "name": "adkosmachev@edu.hse.ru",
      "type": "user"
    }
  }
]


In [31]:
! echo "yes" | terraform apply

azurerm_resource_group.lsml_rg: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group]
azurerm_virtual_network.lsml_vn: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Network/virtualNetworks/lsml-vitrual-network]
azurerm_storage_account.lsml_sa: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.Storage/storageAccounts/lsmlhdinsightstore]
azurerm_storage_container.lsml_hadoop_sc: Refreshing state... [id=https://lsmlhdinsightstore.blob.core.windows.net/lsmlhdinsight]
azurerm_hdinsight_hadoop_cluster.lsml_hc: Refreshing state... [id=/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.HDInsight/clusters/lsml-hdicluster]

An execution plan has been generated and is shown below.
Resource actions are indicated with the f

In [33]:
! terraform output

https_endpoint = "lsml-hdicluster.azurehdinsight.net"
ssh_endpoint = "lsml-hdicluster-ssh.azurehdinsight.net"


Открываем lsml-hdicluster.azurehdinsight.net в браузере и вводим логин\пароль от гейтвея.

Мы попадем в интерфейс Ambari, где можно посмотреть состояние кластера и обновить какие-то параметры.

Мы также можем подключиться к головной машине, используя ssh_endpoint

```bash
ssh azureuser@lsml-hdicluster-ssh.azurehdinsight.net
```

На самой машине можете запустить команду

```bash
hdfs dfs -ls /


Found 18 items
-rwxrwxrwx   1                            0 2021-01-06 15:31 /HDInsight_TestAccessiblityBlobName
drwxr-xr-x   - root   supergroup          0 2021-01-06 16:01 /HdiSamples
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /ams
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /amshbase
drwxrwxrwx   - yarn   hadoop              0 2021-01-06 15:32 /app-logs
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /apps
drwxr-xr-x   - yarn   hadoop              0 2021-01-06 15:32 /atshistory
drwxr-xr-x   - root   supergroup          0 2021-01-06 15:58 /custom-scriptaction-logs
drwxr-xr-x   - root   supergroup          0 2021-01-06 15:58 /example
drwxr-xr-x   - hbase  supergroup          0 2021-01-06 15:32 /hbase
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /hdp
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /hive
drwxr-xr-x   - mapred supergroup          0 2021-01-06 15:32 /mapred
drwxrwxrwx   - mapred hadoop              0 2021-01-06 15:32 /mr-history
drwxrwxrwx   - hdfs   supergroup          0 2021-01-06 15:32 /tmp
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /user
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:32 /warehouse
drwxr-xr-x   - hdfs   supergroup          0 2021-01-06 15:42 /yarn

```

Вам отобразится состояние HDFS хранилища. Важно отметить, что это специальное HDFS хранилище, подключенное к WASB.
Таким образом все результаты работы на кластере автоматически сохраняются в облачное хранилище и вы можете получить к ним доступ не только из Hadoop (проверьте и откройте соответствующий контейнер в браузере). 

Чтобы узнать приватный IP адрес головной машины, к которой нужно подключаться (через прокси) можно запустить команду 

```bash
ifconfig
```

на самой машине. 

Дальнейшие команды семинара нужно будет запускать именно в облаке. Это можно делать просто через терминал, а можно и поднять там Jupyter и работать через него (как это делалось в первом семинаре).

### Загружаем данные в HDFS

**ВАЖНО**: При следующем создании кластера нужно указать ту же самую запись хранения и тот же самый контейнер, чтобы все данные, с которомы вы работали в HDFS сохранились и вы могли продожить с ними работу.

Посмотреть на ваши данные в HDFS можно через BLOB-storage view в самом Azure.

Подгрузим данные с твитами в хадуп

In [78]:
! curl -O https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_{`seq -s , 1 13`}.csv


[1/13]: https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv --> IRAhandle_tweets_1.csv
--_curl_--https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.9M  100 89.9M    0     0  99.3M      0 --:--:-- --:--:-- --:--:-- 99.2M

[2/13]: https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_2.csv --> IRAhandle_tweets_2.csv
--_curl_--https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_2.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.0M  100 90.0M    0     0  16.5M      0  0:00:05  0:00:05 --:--:-- 20.3M

[3/13]: https://raw.githubuse

Подформатируем

In [79]:
! for i in {1..13}; do sed IRAhandle_tweets_$i.csv -i -e '1'd && echo "Finish $i" ; done

Finish 1
Finish 2
Finish 3
Finish 4
Finish 5
Finish 6
Finish 7
Finish 8
Finish 9
Finish 10
Finish 11
Finish 12
Finish 13


Создадим отдельную папку для этих данных в HDFS

In [80]:
! hdfs dfs -ls /

Found 19 items
-rwxrwxrwx   1                               0 2021-01-27 06:46 /HDInsight_TestAccessiblityBlobName
drwxr-xr-x   - root      supergroup          0 2021-01-26 18:51 /HdiSamples
drwxr-xr-x   - hdfs      supergroup          0 2021-01-27 06:47 /ams
drwxr-xr-x   - hdfs      supergroup          0 2021-01-27 06:47 /amshbase
drwxrwxrwx   - yarn      hadoop              0 2021-01-27 06:47 /app-logs
drwxr-xr-x   - hdfs      supergroup          0 2021-01-27 06:47 /apps
drwxr-xr-x   - yarn      hadoop              0 2021-01-27 06:47 /atshistory
drwxr-xr-x   - root      supergroup          0 2021-01-26 18:46 /custom-scriptaction-logs
drwxr-xr-x   - root      supergroup          0 2021-01-26 18:48 /example
drwxr-xr-x   - hbase     supergroup          0 2021-01-27 06:47 /hbase
drwxr-xr-x   - hdfs      supergroup          0 2021-01-27 06:47 /hdp
drwxr-xr-x   - hdfs      supergroup          0 2021-01-27 06:47 /hive
drwxr-xr-x   - mapred    supergroup          0 2021-01-27 06:47 /mapred
d

In [81]:
! hdfs dfs -rm -r /tweets/data
! hdfs dfs -mkdir -p /tweets/data

rm: `/tweets/data': No such file or directory


Note: все команды для hdfs смотреть здесь - https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html

Заливаем данные

In [82]:
! hdfs dfs -put IRAhandle_tweets_* /tweets/data/

21/01/27 12:10:35 WARN impl.MetricsSinkAdapter: azurefs2 has a full queue and can't consume the given metrics.


In [83]:
! hdfs dfs -ls /tweets/data/

Found 13 items
-rw-r--r--   1 azureuser supergroup   94371561 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_1.csv
-rw-r--r--   1 azureuser supergroup   94371615 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_10.csv
-rw-r--r--   1 azureuser supergroup   94371552 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_11.csv
-rw-r--r--   1 azureuser supergroup   94371703 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_12.csv
-rw-r--r--   1 azureuser supergroup    8238864 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_13.csv
-rw-r--r--   1 azureuser supergroup   94371748 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_2.csv
-rw-r--r--   1 azureuser supergroup   94371796 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_3.csv
-rw-r--r--   1 azureuser supergroup   94371606 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_4.csv
-rw-r--r--   1 azureuser supergroup   94371616 2021-01-27 12:10 /tweets/data/IRAhandle_tweets_5.csv
-rw-r--r--   1 azureuser supergroup   94371646 2021-01-27 12:10 /tweets/data/IRAh

### Запускаем MapReduce

Проверяем, что скрипты на головной машине

In [84]:
! cat wordcount.py

import sys
import csv
import re


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv.reader(iter(sys.stdin.readline, '')):
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    word, number = next(sys.stdin).split('\t')
    number = int(number)
    for line in sys.stdin:
        current_word, current_number = line.split('\t')
        current_number = int(current_number)
        if current_word != word:
            print("{}\t{}".format(word, number))
            word = current_word
            number = current_number
        else:
            number += current_number
    print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


In [85]:
! cat top10.py

import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(10):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


Собираем команду на запуск

In [86]:
%%time

! hdfs dfs -rm -r /tweets/result || true
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount.py \
-mapper "python3 wordcount.py map" \
-reducer "python3 wordcount.py reduce" \
-input /tweets/data/ \
-output /tweets/result/

rm: `/tweets/result': No such file or directory
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob2126156697770524973.jar tmpDir=null
21/01/27 12:19:57 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:19:57 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:19:58 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:19:58 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:20:00 INFO client.RequestHedgingRMFailoverProxyProvider: Looking for the active RM in [rm1, rm2]...
21/01/27 12:20:00 INFO client.RequestHedgingRMFailoverProxyProvider: Found active RM [rm2]
21/01/27 12:20:02 INFO mapred.FileInputFormat: Total input files to process : 13
21/01/27 12:20:02 INFO mapreduce.JobSubmitter: number of splits:13
21/01/27 12:20:03 INFO mapreduce.J

Смотрим результат


In [87]:
! hdfs dfs -ls /tweets/result

Found 4 items
-rw-r--r--   1 azureuser supergroup          0 2021-01-27 12:21 /tweets/result/_SUCCESS
-rw-r--r--   1 azureuser supergroup   10107405 2021-01-27 12:21 /tweets/result/part-00000
-rw-r--r--   1 azureuser supergroup   10134121 2021-01-27 12:21 /tweets/result/part-00001
-rw-r--r--   1 azureuser supergroup   10118293 2021-01-27 12:21 /tweets/result/part-00002


In [88]:
! hdfs dfs -cat /tweets/result/part-* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Чтобы посмотреть на консоль хадупа можно
* Добавить *.internal.cloudapp.net в прокси
* Добавить headnodehost в прокси
* Поднять прокси через головную ноду кластера
* Открыть `http://headnodehost:19888/jobhistory`
* Или открыть `http://hn1-hadoop.n3hsvtzmijuexf0fi4yqszkanb.bx.internal.cloudapp.net:8088/cluster` (ссылка может отличаться - она всегда пишется в консоли при запуске MR)

Запустим вторую задачу 

In [89]:
%%time

! hdfs dfs -rm -r /tweets/top10/
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="top-10" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10.py \
-mapper "python top10.py map" \
-reducer "python top10.py reduce" \
-input /tweets/result/ \
-output /tweets/top10/

rm: `/tweets/top10/': No such file or directory
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob2022702067033492020.jar tmpDir=null
21/01/27 12:28:10 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:28:10 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:28:10 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:28:10 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:28:13 INFO client.RequestHedgingRMFailoverProxyProvider: Looking for the active RM in [rm1, rm2]...
21/01/27 12:28:13 INFO client.RequestHedgingRMFailoverProxyProvider: Found active RM [rm2]
21/01/27 12:28:14 INFO mapred.FileInputFormat: Total input files to process : 3
21/01/27 12:28:15 INFO mapreduce.JobSubmitter: number of splits:3
21/01/27 12:28:16 INFO mapreduce.Job

In [90]:
! hdfs dfs -ls /tweets/top10

Found 2 items
-rw-r--r--   1 azureuser supergroup          0 2021-01-27 12:28 /tweets/top10/_SUCCESS
-rw-r--r--   1 azureuser supergroup        106 2021-01-27 12:28 /tweets/top10/part-00000
21/01/27 12:28:59 WARN impl.MetricsSinkAdapter: azurefs2 has a full queue and can't consume the given metrics.


In [91]:
! hdfs dfs -cat /tweets/top10/part-*

t	3015051
co	2833375
https	2454132
the	591885
to	589004
in	457433
a	412888
s	397889
http	375299
of	350983


### Distributed cache

Помимо самого скрипта, мы можем положить в MapReduce любой другой файл, который может пригодиться для работы программы. Например при подсчете количества слов мы бы хотели выкинуть "стоп-слова". Их количество скорее всего не очень большое поэтому смело может передавать их обычным файлом. Hadoop гарантирует, что доставит все файлы ко всем машинам.

In [92]:
! hdfs dfs -cat /tweets/top10/part-* > stop-words.txt

#### Хозяйке на заметку

В питоне уже есть хорошая стандартная библиотека, которая позволяет вам гораздо удобнее работать с такимим стримовыми данными. Давайте напишем новую задачу со стоп словами, чтобы они смотрелись поприличнее.

In [93]:
%%writefile wordcount2.py

import sys
import csv
import re
from itertools import groupby


def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv_stream():
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    for key, group in groupby(kv_stream(), lambda x: x[0]):
        word = key
        number = sum(int(x) for _, x in group)
        print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing wordcount2.py


In [94]:
%%writefile top10-2.py

import sys
import collections
from itertools import islice

def build_stop_words():
    with open('stop-words.txt', 'r') as f:
        stop_words = {x.split('\t')[0] for x in f}
    return stop_words

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for key, value in kv_stream():
        print("{}+{}\t".format(key, value.strip()))

def reducer():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}\t{}".format(word, count.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing top10-2.py


In [33]:
! cat stop-words.txt

t	3015051
co	2833375
https	2454132
the	591885
to	589004
in	457433
a	412888
s	397889
http	375299
of	350983


In [95]:
%%time

! hdfs dfs -rm -r /tweets/top10-stop-words/
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="top-10-stop-words" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10-2.py,stop-words.txt \
-mapper "python top10-2.py map" \
-reducer "python top10-2.py reduce" \
-input /tweets/result/ \
-output /tweets/top10-stop-words/

rm: `/tweets/top10-stop-words/': No such file or directory
21/01/27 12:38:10 WARN impl.MetricsSinkAdapter: azurefs2 has a full queue and can't consume the given metrics.
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob6652079870412143204.jar tmpDir=null
21/01/27 12:38:20 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:38:20 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:38:20 ERROR sender.RawSocketSender: org.fluentd.logger.sender.RawSocketSender
java.net.SocketTimeoutException
	at java.net.SocksSocketImpl.remainingMillis(SocksSocketImpl.java:111)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at org.fluentd.logger.sender.RawSocketSender.connect(RawSocketSender.java:85)
	at org.fluentd.logger.sender.RawSocketSender.reconnect(RawSocketSender.java:94)
	at org.fluentd.logge

In [96]:
! hdfs dfs -cat /tweets/top10-stop-words/*

i	287232
for	272995
and	247749
is	246856
on	210172
you	196950
trump	169520
news	156101
it	152816
with	134178


### Ускоряем вычисления 

Несмотря на все оптимизации внутри Hadoop, самое узкое место - это передача данных от mapper к reducer. Таким образом если у нас получиться ускорить выполнение .

In [97]:
%%writefile wordcount3.py

import sys
import csv
import re
from itertools import groupby
from collections import Counter


def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)


def mapper():
    counter = Counter()
    pattern = re.compile(r"[a-z]+")
    for row in csv_stream():
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            counter[word] += 1
    
    for word, number in counter.items():
        print("{}\t{}".format(word, number))


def reducer():
    for key, group in groupby(kv_stream(), lambda x: x[0]):
        word = key
        number = sum(int(x) for _, x in group)
        print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing wordcount3.py


In [98]:
%%time

! hdfs dfs -rm -r /tweets/result-fast1 || true
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount3.py \
-mapper "python3 wordcount3.py map" \
-reducer "python3 wordcount3.py reduce" \
-input /tweets/data/ \
-output /tweets/result-fast1/

rm: `/tweets/result-fast1': No such file or directory
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob3276516031445984988.jar tmpDir=null
21/01/27 12:47:50 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:47:50 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:47:50 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:47:50 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:47:52 INFO client.RequestHedgingRMFailoverProxyProvider: Looking for the active RM in [rm1, rm2]...
21/01/27 12:47:53 INFO client.RequestHedgingRMFailoverProxyProvider: Found active RM [rm2]
21/01/27 12:47:54 INFO mapred.FileInputFormat: Total input files to process : 13
21/01/27 12:47:54 INFO mapreduce.JobSubmitter: number of splits:13
21/01/27 12:47:54 INFO mapre

In [99]:
! hdfs dfs -cat /tweets/result-fast1/* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Однако у этого решения есть **очень большой минус** - сложность по памяти **O(n)**. Это означает, что вычисление может упасть если данные попадутся неудачные. 

Важный принцип работы с большими данными - все алгоритмы должны работать меньше чем за O(n). Это относится не только к MapReduce, а в целом почти к любым инструментам обработки больших данных.

### Используем комбайнер

Чтобы побороться с этой бедой, воспользуемся дополнительным инструментом в Hadoop - Combiner. По сути это маленький Reduce, который запускается после маппера. Это позволяет уменьшить количество выходных данных с Map стадии.

<img src="https://habrastorage.org/getpro/habr/post_images/587/2d2/dfe/5872d2dfe12643665370708d225bc1d4.jpg">

In [100]:
%%time

! hdfs dfs -rm -r /tweets/result-fast2 || true
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount2.py \
-mapper "python3 wordcount2.py map" \
-combiner "python3 wordcount2.py reduce" \
-reducer "python3 wordcount2.py reduce" \
-input /tweets/data/ \
-output /tweets/result-fast2/

rm: `/tweets/result-fast2': No such file or directory
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob8321391744385464559.jar tmpDir=null
21/01/27 12:55:34 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:55:34 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:55:34 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 12:55:34 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 12:55:36 INFO client.RequestHedgingRMFailoverProxyProvider: Looking for the active RM in [rm1, rm2]...
21/01/27 12:55:36 INFO client.RequestHedgingRMFailoverProxyProvider: Found active RM [rm2]
21/01/27 12:55:38 INFO mapred.FileInputFormat: Total input files to process : 13
21/01/27 12:55:38 INFO mapreduce.JobSubmitter: number of splits:13
21/01/27 12:55:38 INFO mapre

In [101]:
! hdfs dfs -cat /tweets/result-fast1/* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Часто combiner может просто совпадать с reducer однако это не всегда так по следующей причине - combiner не имеет права менять формат вывода после стадии map.

Hadoop самостоятельно опеределяет целесообразность запуска combiner и может его не запускать вовсе.
Или например задача может вообще не подходить под такую модель запуска. Если мы ищем среднее, то нельзя заранее подсчитывать среднее на стадии combiner - макмимум, что мы там можем запустить - это подсчет количество и суммы.

In [102]:
%%writefile top10-3.py

import sys
import collections
from itertools import islice

def build_stop_words():
    with open('stop-words.txt', 'r') as f:
        stop_words = {x.split('\t')[0] for x in f}
    return stop_words

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for key, value in kv_stream():
        print("{}+{}\t".format(key, value.strip()))

def reducer():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}\t{}".format(word, count.strip()))
    rewind()
    
def combiner():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}+{}\t".format(word, count.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer,
        'combiner': combiner
    }[mr_command]()

Writing top10-3.py


In [103]:
%%time

! hdfs dfs -rm -r /tweets/top10-fast || true
! yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/top10-3.py,stop-words.txt \
-mapper "python3 top10-3.py map" \
-combiner "python3 top10-3.py combiner" \
-reducer "python3 top10-3.py reduce" \
-input /tweets/result-fast1 \
-output /tweets/top10-fast/

rm: `/tweets/top10-fast': No such file or directory
packageJobJar: [] [/usr/hdp/4.1.2.5/hadoop/hadoop-streaming-3.1.3.4.1.2.5.jar] /tmp/streamjob6007250661757710875.jar tmpDir=null
21/01/27 13:04:07 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 13:04:07 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 13:04:07 INFO client.RequestHedgingRMFailoverProxyProvider: Created wrapped proxy for [rm1, rm2]
21/01/27 13:04:07 INFO client.AHSProxy: Connecting to Application History server at headnodehost/10.0.0.21:10200
21/01/27 13:04:09 INFO client.RequestHedgingRMFailoverProxyProvider: Looking for the active RM in [rm1, rm2]...
21/01/27 13:04:09 INFO client.RequestHedgingRMFailoverProxyProvider: Found active RM [rm2]
21/01/27 13:04:10 INFO mapred.FileInputFormat: Total input files to process : 3
21/01/27 13:04:11 INFO mapreduce.JobSubmitter: number of splits:3
21/01/27 13:04:11 INFO mapreduce

In [104]:
! hdfs dfs -cat /tweets/top10-fast/* 

i	287232
for	272995
and	247749
is	246856
on	210172
you	196950
trump	169520
news	156101
it	152816
with	134178


#### Самостоятельное упражнение для искушенного слушателя

Давайте теперь мы попробуем обработать скачанные выше данные по-взрослому. Пусть у нас есть какой-то SQL движок (здесь и далее будет использоваться синтаксис Clickhouse), в котором есть вот такая таблица:

```sql
create table tweet_data (
    external_author_id   String,
    author               String,
    content              String,
    region               String,
    language             String,
    publish_date         String,
    harvested_date       String,
    following            String,
    followers            String,
    updates              String,
    post_type            String,
    account_type         String,
    retweet              String,
    account_category     String,
    new_june_2018        String,
    alt_external_id      String,
    tweet_id             String,
    article_url          String,
    tco1_step1           String,
    tco2_step1           String,
    tco3_step1           String
)
engine = MergeTree()
    order by author
```

Для особых ценителей:
```bash
python -c 'import csv; data=open("tweets_1.csv", newline=""); print("\n".join(map(lambda line: "|||".join(map(lambda rawline: rawline.replace("\t", ""), line)), csv.reader(data))));' \
    | tail -n+2 \
    | sed 's:\\:\\\\:g' \
    | sed 's:\t: :g' \
    | sed 's:|||:\t:g' \
    | clickhouse local \
        --input-format TSV \
        --table tmp \
        --structure 'external_author_id String, author String, content String, region String, language String, publish_date String, harvested_date String, following String, followers String, updates String, post_type String, account_type String, retweet String, account_category String, new_june_2018 String, alt_external_id String, tweet_id String, article_url String, tco1_step1 String, tco2_step1 String, tco3_step1 String'\
        --query "select * from tmp limit 1"
```

Чтобы было удобнее работать с этими данным ad-hoc лучше всего предобработать данные и создать временный `alias` в каком-то терминале:

```bash
python -c 'import csv; data=open("tweets_1.csv", newline=""); print("\n".join(map(lambda line: "|||".join(map(lambda rawline: rawline.replace("\t", ""), line)), csv.reader(data))));' \
    | tail -n+2 \
    | sed 's:\\:\\\\:g' \
    | sed 's:\t: :g' \
    | sed 's:|||:\t:g' > tweets_1_treated.csv
    
alias tmpch="cat tweets_1_treated.csv \
    | clickhouse local \
        --input-format TSV \
        --table tmp \
        --structure 'external_author_id String, author String, content String, region String, language String, publish_date String, harvested_date String, following String, followers String, updates String, post_type String, account_type String, retweet String, account_category String, new_june_2018 String, alt_external_id String, tweet_id String, article_url String, tco1_step1 String, tco2_step1 String, tco3_step1 String'\
        --query"
```

К нам поступает задача: посчитать топ-10 пользователей, которые написали больше 100 твитов в разбивке по регионам. Примерное решение задачи может выглядеть следующим образом:


```sql
select region, 
       sum(num_tweets) as num_tweets, 
       arrayStringConcat(arraySlice(groupArray(author), 1, 10), ', ') 
from (
    select author, 
           region, 
           count() as num_tweets
    from tmp
    group by author, region
    having num_tweets > 100
    order by num_tweets DESC
) 
group by region 
order by num_tweets DESC
```

Давайте посмотрим на стадии выполнения запроса:
1. Выбираем все пары пользователь-регион, у которых сумма твитов > 100
2. Сортируем в порядке убывания количества твитов
3. Группируем по региону, считаем сумму пользователей
4. Для каждого региона берем 10 первых авторов и перечисляем их ники через запятую

Подумайте, как бы вы это реализовали на map-reduce?

Какой шаг здесь лучше всего?

1. Здесь будет следующая последовательность map-reduce-map-reduce: шаг map прочитает файл и отфильтрует ненужные колонки, шаг reduce -- посчитает по каждой паре пользователь-региону сумму твитов, map отфильтрует среди них те, у которых твитов больше 100 и переформатирует данные так, что они приумт вид `f"{num_tweets} {author} {region}"`, а последняя функция reduce будет identity-фнукцией
2. Этот шаг мы можем не делать -- на выходе из предыдущего шага ключ-значение уже дадут нам нужную сортировку
3. Здесь будет map-reduce: шаг map преобразует файл в `f"{region} {author} {num_tweets}"`, затем reduce соберет по каждому региону суммарную статистику и выдаст в результате файл с регионом, числом твитов и пользователями, где каждая строка -- свой пользователь
4. Здесь будет map-reduce: шаг map будет ничего не делать, а шаг reduce соберет нужную нам строчку для каждого региона.

Решение задачи может выглядеть следующим образом:

In [37]:
%%writefile mapper-projection.py
import sys
import csv

def main():
    for row in csv.reader(iter(sys.stdin.readline, '')):
        current_row = [f"{row[1]}_{[3]}", 1]
        print("\t".join(current_row))
        
if __name__ == "__main__":
    main()

Writing mapper-projection.py


In [38]:
%%writefile reducer-groupby-sum.py
import collections
import sys
import csv

def main():
    groupby = collections.defaultdict(int)
    
    for (author_region, tweet_count) in csv.reader(iter(sys.stdin.readline, ''), delimiter='\t'):
        tweet_count = int(tweet_count)
        groupby[author_region] += tweet_count
        
    for (author_region, tweet_count) in groupby:
        print(f"{author_region}\t{tweet_count}")
        
if __name__ == "__main__":
    main()

Writing reducer-groupby-sum.py


In [39]:
%%writefile mapper-filter.py
import sys
import csv

def main():
    for (author_region, tweet_count) in csv.reader(iter(sys.stdin.readline, ''), delimiter='\t'):
        author, region = author_region.split("_")
        if int(tweet_count) > 100:
            print(f"{tweet_count}\t{author}\t{region}")
        
if __name__ == "__main__":
    main()


Writing mapper-filter.py


In [40]:
%%writefile identity.py
def main():
    for line in iter(sys.stdin.readline, ''):
        print(line)
        
if __name__ == "__main__":
    main()

Writing identity.py


In [41]:
# DO!: допишите код так, чтобы он заработал

### Hadoop жжет бабло

<img src="http://vostokovod.ru/assets/images/blog/2013/000333.png">

Полноценный кластер - весьма дорогое удовольствие, поэтому отлючайте его после использования. 

**Если в HDFS у вас лежит какой-то результат, который вы не хотите терять - не удаляйте `storage container`!**. К новому кластеру можно будет подключить сохраненный контейнер и продолжить работу.

In [35]:
# Удаляем ТОЛЬКО кластер
! echo "yes" | terraform destroy -target azurerm_hdinsight_hadoop_cluster.lsml_hc 


An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_hdinsight_hadoop_cluster.lsml_hc will be destroyed
  - resource "azurerm_hdinsight_hadoop_cluster" "lsml_hc" {
      - cluster_version     = "4.0.2000.1" -> null
      - https_endpoint      = "lsml-hdicluster.azurehdinsight.net" -> null
      - id                  = "/subscriptions/7d1225ca-27cc-40b7-8036-c62a48072ba8/resourceGroups/lsml-resource-group/providers/Microsoft.HDInsight/clusters/lsml-hdicluster" -> null
      - location            = "westus" -> null
      - name                = "lsml-hdicluster" -> null
      - resource_group_name = "lsml-resource-group" -> null
      - ssh_endpoint        = "lsml-hdicluster-ssh.azurehdinsight.net" -> null
      - tags                = {} -> null
      - tier                = "standard" -> null

      - component_version {
          - hadoop = "3.1" -> n